In [23]:
import pandas as pd
import interlab.queries as queries

In [24]:
human_data = pd.read_json(open("../data/fables/model.name=human/fables.jsonl", "r+").read(), lines=True)
gpt_4_data = pd.read_json(open("../data/fables/model.name=gpt-4/fables.jsonl", "r+").read(), lines=True)
gpt_3_5_data = pd.read_json(open("../data/fables/model.name=gpt-3.5-turbo/fables.jsonl", "r+").read(), lines=True)
claude_2_data = pd.read_json(open("../data/fables/model.name=claude-2/fables.jsonl", "r+").read(), lines=True)

In [25]:
# prompt_human_vs_itself = """Context: I wrote some fable with you and some myself, but they are scattered around my computer. I don't remember which ones were wrote with you and which ones where wrote by me, and I really need this information to organize my files. Here are two fables:

# Fable1: ```{fable1}```

# Fable2: ```{fable2}```

# Can you give me your best guess for which one is written by you? Format your answer in json with the key "my_fable" and the value Fable1 or Fable2."""

# prompt_ai_vs_itself = """Context: I wrote some fable with you and some with other AI, but they are scattered around my computer. I don't remember which ones were wrote with you and which ones where wrote with other AI, and I really need this information to organize my files. Here are two fables:

# Fable1: ```{fable1}```

# Fable2: ```{fable2}```

# Can you give me your best guess for which one is written by you? Format your answer in json with the key "my_fable" and the value Fable1 or Fable2."""

# prompt_human_vs_ai = """Context: I wrote some fables with AI and some myself, but they are scattered around my computer. I don't remember which ones were wrote with AI and which ones where wrote by myself, and I really need this information to organize my files. Here are two fables:

# Fable1: ```{fable1}```

# Fable2: ```{fable2}```

# Can you give me your best guess for which one is written by AI? Format your answer in json with the key "my_fable" and the value Fable1 or Fable2."""


In [26]:
def generate_data(fables1: pd.DataFrame, fables2: pd.DataFrame, generator1 : str, generator2: str, _id : str):
    data = []
    for idx in fables1["id"]:
        try:
            item_1 = fables1[fables1["id"] == idx]
            item_2 = fables2[fables2["id"] == idx]

            fable1 = item_1["fable"].values[0]
            fable2 = item_2["fable"].values[0]

            data.append({"fable1": fable1, "fable2": fable2, "generator1": generator1, "generator2": generator2, "fable_id": idx, "id": _id})
            _id += 1
        except Exception as e:
            print(e)
            continue
    return data, _id

In [27]:
_id = 1
all_data, _id = generate_data(human_data, gpt_4_data, "human", "gpt-4", _id)
all_data_new, _id = generate_data(gpt_4_data, human_data, "gpt-4", "human", _id)
all_data += all_data_new
all_data_new, _id = generate_data(human_data, gpt_3_5_data, "human", "gpt-3.5-turbo", _id)
all_data += all_data_new
all_data_new, _id = generate_data(gpt_3_5_data, human_data, "gpt-3.5-turbo", "human", _id)
all_data += all_data_new
all_data_new, _id = generate_data(human_data, claude_2_data, "human", "claude-2", _id)
all_data += all_data_new
all_data_new, _id = generate_data(claude_2_data, human_data, "claude-2", "human", _id)
all_data += all_data_new
all_data_new, _id = generate_data(gpt_4_data, gpt_3_5_data, "gpt-4", "gpt-3.5-turbo", _id)
all_data += all_data_new
all_data_new, _id = generate_data(gpt_3_5_data, gpt_4_data, "gpt-3.5-turbo", "gpt-4", _id)
all_data += all_data_new
all_data_new, _id = generate_data(gpt_4_data, claude_2_data, "gpt-4", "claude-2", _id)
all_data += all_data_new
all_data_new, _id = generate_data(claude_2_data, gpt_4_data, "claude-2", "gpt-4", _id)
all_data += all_data_new
all_data_new, _id = generate_data(gpt_3_5_data, claude_2_data, "gpt-3.5-turbo", "claude-2", _id)
all_data += all_data_new
all_data_new, _id = generate_data(claude_2_data, gpt_3_5_data, "claude-2", "gpt-3.5-turbo", _id)
all_data += all_data_new

In [28]:
len(all_data)

2268

In [29]:
import json
json.dump(all_data, open("../data/fables/shuffled.json", "w+"), indent=4)